In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

from sklearn.model_selection import train_test_split

import numpy as np
from matplotlib import pyplot as plt
import glob
import cv2

In [ ]:
def prepare_data(mode):
    """ divides data into images and labels
        Min-Max scales images
    Params:
        mode : String (either "padded" or "resized")
    Returns:
        scaled_X : array with images
        y : array with corresponding labels
        (0 for non-IDC, 1 for IDC)
    """
    patches = glob.glob('./' + mode + '/*.png', recursive=True)
    class0 = []
    class1 = []
    for name in glob.glob('./' + mode + '/*class0.png', recursive=True):
        class0.append(name)
    for name in glob.glob('./' + mode + '/*class1.png', recursive=True):
        class1.append(name)
    X = []
    y = []
    for img_name in patches:
        image = cv2.imread(img_name)
        X.append(image)
        y.append(0) if img_name in class0 else y.append(1)
    scaled_X = np.array(X)/255.0
    return scaled_X, y

In [ ]:
X_pad, y = prepare_data("padded")
X_inter, _ = prepare_data("resized")

In [ ]:
X_pad_train, X_pad_test, y_pad_train, y_pad_test = train_test_split(X_pad, y, test_size=0.1, random_state=42)
X_inter_train, X_inter_test, y_inter_train, y_inter_test = train_test_split(X_inter, y, test_size=0.1, random_state=42)

In [ ]:
y_pad_train_onehot = np_utils.to_categorical(y_pad_train, 2)
y_pad_test_onehot = np_utils.to_categorical(y_pad_test, 2)
y_inter_train_onehot = np_utils.to_categorical(y_inter_train, 2)
y_inter_test_onehot = np_utils.to_categorical(y_inter_test, 2)

In [ ]:
def create_CNN():
    input_shape = (50, 50, 3)
    num_classes = 2
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape)) # strides=(1, 1) default
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
epochs = 10
batch_size = 128

cnn_pad = create_CNN()
cnn_pad.summary()

In [ ]:
pad_fit = cnn_pad.fit(X_pad_train, y_pad_train_onehot,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

In [ ]:
score_pad = cnn_pad.evaluate(X_pad_test, y_pad_test_onehot, verbose=0)
print('Test accuracy:', score_pad[1])

plt.plot(pad_fit.history['loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Simple CNN with padded imgs')
plt.show()

In [ ]:
# for comparison with resized imgs
cnn_inter = create_CNN()
inter_fit = cnn_inter.fit(X_inter_train, y_inter_train_onehot,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

In [ ]:
score_inter = cnn_inter.evaluate(X_inter_test, y_inter_test_onehot, verbose=0)
print('Test accuracy:', score_inter[1])

plt.plot(inter_fit.history['loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Simple CNN with interpolated imgs')
plt.show()